### Required imports

In [ ]:
from firebolt.db import connect, OperationalError
from firebolt.client import DEFAULT_API_URL
from firebolt.client.auth import ClientCredentials
from datetime import datetime

### Database credentials

In [ ]:
client_id = ""
client_secret = ""
account_name = ""
engine_name = ""  # Optional
database_name = ""  # Optional
api_endpoint = DEFAULT_API_URL  # Optional

### Connecting to a database and creating cursor

In [ ]:
# create a connection based on provided credentials
connection = connect(
    auth=ClientCredentials(client_id, client_secret),
    account_name=account_name,
    engine_name=engine_name,
    database=database_name,
    api_endpoint=api_endpoint,
)

# create a cursor for connection
cursor = connection.cursor()

### Executing a query

In [ ]:
cursor.execute(
    "create fact table if not exists test_table (id int, name text, dt datetime) primary index id"
)
cursor.execute(
    "insert into test_table values (1, 'hello', '2021-01-01 01:01:01'),"
    "(2, 'world', '2022-02-02 02:02:02'),"
    "(3, '!', '2023-03-03 03:03:03')"
)

### Parameterized query

In [ ]:
cursor.execute(
    "insert into test_table values (?, ?, ?)",
    (3, "single parameter set", datetime.now()),
)
cursor.executemany(
    "insert into test_table values (?, ?, ?)",
    ((4, "multiple", datetime.now()), (5, "parameter sets", datetime.fromtimestamp(0))),
)

### Getting query description, rowcount

In [ ]:
cursor.execute("select * from test_table")
print("Description: ", cursor.description)
print("Rowcount: ", cursor.rowcount)

### Fetch query results

In [ ]:
print(cursor.fetchone())
print(cursor.fetchmany(1))
print(cursor.fetchall())

## Multi-statement queries

In [ ]:
cursor.execute(
    """
    select * from test_table where id < 4;
    select * from test_table where id > 2;
"""
)
print("First query: ", cursor.fetchall())
assert cursor.nextset()
print("Second query: ", cursor.fetchall())
assert cursor.nextset() is None

## Error handling
If one query fails during the execution, all remaining queries are canceled.
However, you still can fetch results for successful queries

In [ ]:
try:
    cursor.execute(
        """
        select * from test_table where id < 4;
        select * from test_table where wrong_field > 2;
        select * from test_table
    """
    )
except OperationalError:
    pass
cursor.fetchall()

## Query result streaming

Streaming is useful for large result sets, when you want to process rows one by one without loading all of them into memory.

In [ ]:
cursor.execute_stream("select * from generate_series(1, 1000000)")
for row in cursor:
    print(row)
    if row[0] > 10:
        break
# Remaining rows will not be fetched

## Async interface
**NOTE**: In order to make async examples work in jupyter, you would need to install [trio-jupyter](https://github.com/mehaase/trio-jupyter) library and select **Python 3 Trio** kernel

In [ ]:
from firebolt.async_db import connect as async_connect

### Connecting to a database and creating cursor

In [ ]:
# create a connection based on provided credentials
async_connection = await async_connect(
    auth=ClientCredentials(client_id, client_secret),
    account_name=account_name,
    engine_name=engine_name,
    database=database_name,
    api_endpoint=api_endpoint,
)

# create a cursor for connection
async_cursor = async_connection.cursor()

### Executing a query

In [ ]:
await async_cursor.execute(
    "create fact table if not exists test_table (id int, name text, dt datetime) primary index id"
)
await async_cursor.execute(
    "insert into test_table values (1, 'hello', '2021-01-01 01:01:01'),"
    "(2, 'world', '2022-02-02 02:02:02'),"
    "(3, '!', '2023-03-03 03:03:03')"
)
await async_cursor.execute("select * from test_table")

### Getting query description, rowcount

In [ ]:
print("Description: ", async_cursor.description)
print("Rowcount: ", async_cursor.rowcount)

### Fetch query results

In [ ]:
print(await cursor.fetchone())
print(await cursor.fetchmany(1))
print(await cursor.fetchall())

### Closing connection

In [ ]:
# manually
connection.close()
print(connection.closed)

await async_connection.aclose()
print(async_connection.closed)

# using context manager
with connect(
    auth=ClientCredentials(client_id, client_secret),
    account_name=account_name,
    engine_name=engine_name,
    database=database_name,
    api_endpoint=api_endpoint,
) as conn:
    # create cursors, perform database queries
    pass
print(conn.closed)

# using context manager
async with await async_connect(
    auth=ClientCredentials(client_id, client_secret),
    account_name=account_name,
    engine_name=engine_name,
    database=database_name,
    api_endpoint=api_endpoint,
) as async_conn:
    # create cursors, perform database queries
    pass
async_conn.closed

## Query result streaming

In [ ]:
await cursor.execute_stream("select * from generate_series(1, 1000000)")
async for row in cursor:
    print(row)
    if row[0] > 10:
        break
# Remaining rows will not be fetched